# Loading Team State from a File

In [1]:
import asyncio
from autogen_core import CancellationToken
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [6]:
import json

with open('team_state.json', 'r') as f:
    team_state = json.load(f)

team_state

{'type': 'TeamState',
 'version': '1.0.0',
 'agent_states': {'Writer_1': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state': {'type': 'AssistantAgentState',
    'version': '1.0.0',
    'llm_context': {'messages': [{'content': 'Write a poem about the sea in 3 lines',
       'source': 'user',
       'type': 'UserMessage'},
      {'content': 'Endless waves whisper,  \nSapphire dreams and salt-kissed air,  \nSecrets in the deep.',
       'thought': None,
       'source': 'Writer_1',
       'type': 'AssistantMessage'}]}},
   'message_buffer': [{'source': 'Writer_2',
     'models_usage': {'prompt_tokens': 66, 'completion_tokens': 22},
     'metadata': {},
     'content': 'Endless waves whisper,  \nSapphire dreams in salt-kissed air,  \nSecrets in the deep.',
     'type': 'TextMessage'}]},
  'Writer_2': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state': {'type': 'AssistantAgentState',
    'version': '1.0.0',
    'llm_context': {'messages': [{

In [7]:
from autogen_agentchat.conditions import MaxMessageTermination

agent_1 = AssistantAgent(
    name='Writer_1',
    model_client=model_client,
    system_message="You are a helpful assistant. Give the output in less than 30 words",
)

agent_2 = AssistantAgent(
    name='Writer_2',
    model_client=model_client,
    system_message="You are a helpful assistant.Give the output in less than 30 words",
)

terminationCondition = MaxMessageTermination(max_messages=3)

new_agent_team = RoundRobinGroupChat(participants=[agent_1, agent_2],termination_condition=terminationCondition)

In [8]:
await new_agent_team.load_state(team_state)

In [9]:
from autogen_agentchat.ui import Console

stream = new_agent_team.run_stream(task="What was the last line of Poem you wrote?")

await Console(stream)

---------- user ----------
What was the last line of Poem you wrote?
---------- Writer_1 ----------
"Secrets in the deep."
---------- Writer_2 ----------
"Secrets in the deep."


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last line of Poem you wrote?', type='TextMessage'), TextMessage(source='Writer_1', models_usage=RequestUsage(prompt_tokens=111, completion_tokens=7), metadata={}, content='"Secrets in the deep."', type='TextMessage'), TextMessage(source='Writer_2', models_usage=RequestUsage(prompt_tokens=125, completion_tokens=7), metadata={}, content='"Secrets in the deep."', type='TextMessage')], stop_reason='Maximum number of messages 3 reached, current message count: 3')